In [330]:
import pandas as pd
import json

In [331]:
# dataset = "snli_dev.txt"
preds_file = "report/Performance-report/model_mnli_train/eval_predictions.jsonl"
source_file = "report/Performance-report/model_snli_train/eval_predictions.jsonl"
preds = open(preds_file, 'r')
source = open(source_file, 'r')
# source = pd.read_csv(dataset, sep="\t")
# source.head()
min_support = 20
min_conditional = 0.8

In [332]:
word_counts = {}
word_counts_correct = {}
word_counts_correct_train = {}
word_count_old = {}
joints = {"neutral_data": {}, "entailment_data": {}, "contradiction_data": {}, "neutral_pred": {}, "entailment_pred": {}, "contradiction_pred": {}, "neutral_train_data": {}, "entailment_train_data": {}, "contradiction_train_data": {}, "neutral_train_pred": {}, "entailment_train_pred": {}, "contradiction_train_pred": {}}
label_mapping = {0: "entailment", 1: "neutral", 2: "contradiction"}

#  (label 0 is "entailed", 1 is "neutral", and 2 is "contradiction")

def add_word(w, d):
    if w in d:
        d[w] += 1
    else:
        d[w] = 1

In [333]:
The guy in a striped shirt is playing his electric guitar. A couple at an outdoor concert.

SyntaxError: invalid syntax (Temp/ipykernel_13248/1540639762.py, line 1)

In [ ]:
A few people in a restaurant setting, one of them is drinking orange juice. The people are eating omelettes.

In [ ]:
A few people in a restaurant setting, one of them is drinking orange juice

In [334]:
for line in source.read().split("\n"):
    try:
        line = json.loads(line)
        # print(line)
        # sample, pred, actual = line["premise"] + " " + line["hypothesis"], label_mapping[line["predicted_label"]], label_mapping[line["label"]]
        sample, pred, actual = line["hypothesis"], label_mapping[line["predicted_label"]], label_mapping[line["label"]]
        for word in sample.split(" "):
            word = word.replace(".", "")
            add_word(word, word_count_old)
            add_word(word, joints[pred + "_train_pred"])
            add_word(word, joints[actual + "_train_data"])
        if pred == actual:
                add_word(word, word_counts_correct_train)
        if pred == "contradiction" and actual == "neutral":
            print(line["premise"], line["hypothesis"])
    # subset = source[(source["premise"] == line['premise']) & (source["hypothesis"] == line['hypothesis'])]
    # if subset.shape[0] < 1:
    #     print("not found")
    except Exception as e:
        print(line, e)

A few people in a restaurant setting, one of them is drinking orange juice. The people are eating omelettes.
Bicyclists waiting at an intersection. The bicyclists are riding to the book store.
three bikers stop in town. cars stop in town
three bikers stop in town. Three bicyclists are riding in a pack.
A family with a baby, the father is wearing a save the children sign. A woman is holding a baby.
One man sits inside and plays the banjo, there are trees behind him outside. A male plays a banjo while sitting on a sofa.
Two young girls are playing outside in a non-urban environment. Some girls are running through an open field.
Three dogs in different shades of brown and white biting and licking each other. the dog playing
A woman is making a clay pot. A woman is making a flower pot.
A little boy swimming underwater with a toy in his hand. A little boy is holding a pool ring.
A baseball player is putting all his might in to throwing a ball. Player is giving the ball away
A camera crew is

In [288]:
for line in preds.read().split("\n"):
    try:
        line = json.loads(line)
        # print(line)
        # sample, pred, actual = line["sentence1"] + " " + line["sentence2"], label_mapping[line["predicted_label"]], line["gold_label"]
        sample, pred, actual = line["sentence2"], label_mapping[line["predicted_label"]], line["gold_label"]
        for word in sample.split(" "):
            word = word.replace(".", "")
            add_word(word, word_counts)
            add_word(word, joints[pred + "_pred"])
            add_word(word, joints[actual + "_data"])
            if pred == actual:
                add_word(word, word_counts_correct)
    # subset = source[(source["premise"] == line['premise']) & (source["hypothesis"] == line['hypothesis'])]
    # if subset.shape[0] < 1:
    #     print("not found")
    except Exception as e:
        print(line, e)
    # break

 Expecting value: line 1 column 1 (char 0)


In [289]:
words = pd.DataFrame.from_dict(word_counts, orient='index').rename(columns={0: "count_val"})
words = words.join(pd.DataFrame.from_dict(word_count_old, orient='index').rename(columns={0: "count_train"}), how="outer")
words = words.join(pd.DataFrame.from_dict(word_counts_correct, orient='index').rename(columns={0: "word_counts_correct"}), how="outer")
words = words.join(pd.DataFrame.from_dict(word_counts_correct_train, orient='index').rename(columns={0: "word_counts_correct_train"}), how="outer")
words["word_counts_correct"] = words["word_counts_correct"].fillna(0)
words.head()

,count_val,count_train,word_counts_correct,word_counts_correct_train
,59010.0,2339.0,41095.0,156.0
!,1.0,1.0,0.0,1.0
!995's,1.0,NaN,1.0,NaN
"""",14.0,NaN,10.0,NaN
"""'Tis",1.0,NaN,1.0,NaN


In [290]:
words = words.join(pd.DataFrame.from_dict(joints["neutral_data"], orient='index').rename(columns={0: "neutral_data"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["contradiction_data"], orient='index').rename(columns={0: "contradiction_data"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["entailment_data"], orient='index').rename(columns={0: "entailment_data"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["neutral_pred"], orient='index').rename(columns={0: "neutral_pred"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["contradiction_pred"], orient='index').rename(columns={0: "contradiction_pred"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["entailment_pred"], orient='index').rename(columns={0: "entailment_pred"}), how="outer")

words = words.join(pd.DataFrame.from_dict(joints["neutral_train_data"], orient='index').rename(columns={0: "neutral_train_data"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["contradiction_train_data"], orient='index').rename(columns={0: "contradiction_train_data"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["entailment_train_data"], orient='index').rename(columns={0: "entailment_train_data"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["neutral_train_pred"], orient='index').rename(columns={0: "neutral_train_pred"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["contradiction_train_pred"], orient='index').rename(columns={0: "contradiction_train_pred"}), how="outer")
words = words.join(pd.DataFrame.from_dict(joints["entailment_train_pred"], orient='index').rename(columns={0: "entailment_train_pred"}), how="outer")
words.head()

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
,59010.0,2339.0,41095.0,156.0,19308.0,20822.0,18880.0,17678.0,18993.0,22339.0,840.0,615.0,884.0,686.0,610.0,1043.0
!,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
!995's,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""",14.0,NaN,10.0,NaN,4.0,5.0,5.0,6.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
"""'Tis",1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [291]:
words["neutral_data"] = words["neutral_data"] / words["count_val"]
words["contradiction_data"] = words["contradiction_data"] / words["count_val"]
words["entailment_data"] = words["entailment_data"] / words["count_val"]
words["neutral_pred"] = words["neutral_pred"] / words["count_val"]
words["contradiction_pred"] = words["contradiction_pred"] / words["count_val"]
words["entailment_pred"] = words["entailment_pred"] / words["count_val"]

words["neutral_train_data"] = words["neutral_train_data"] / words["count_train"]
words["contradiction_train_data"] = words["contradiction_train_data"] / words["count_train"]
words["entailment_train_data"] = words["entailment_train_data"] / words["count_train"]
words["neutral_train_pred"] = words["neutral_train_pred"] / words["count_train"]
words["contradiction_train_pred"] = words["contradiction_train_pred"] / words["count_train"]
words["entailment_train_pred"] = words["entailment_train_pred"] / words["count_train"]
words.head()

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
,59010.0,2339.0,41095.0,156.0,0.327199,0.352855,0.319946,0.299576,0.321861,0.378563,0.359128,0.262933,0.377939,0.293288,0.260795,0.445917
!,1.0,1.0,0.0,1.0,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1.000000
!995's,1.0,NaN,1.0,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""",14.0,NaN,10.0,NaN,0.285714,0.357143,0.357143,0.428571,0.285714,0.285714,NaN,NaN,NaN,NaN,NaN,NaN
"""'Tis",1.0,NaN,1.0,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
print(words.shape)
words = words[words["count_train"] > min_support]
words.shape

(116177, 16)


(5842, 16)

In [316]:
subset = words[(words["neutral_train_pred"] > min_conditional) | (words["contradiction_train_pred"] > min_conditional) | (words["entailment_train_pred"] > min_conditional)]
# subset = subset[(subset["neutral_train_data"] > 0.5) | (subset["contradiction_train_data"] > 0.5) | (subset["entailment_train_data"] > 0.5)]
subset.head(10)

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
4th,18.0,101.0,13.0,4.0,0.388889,0.111111,0.500000,0.388889,0.111111,0.500000,0.871287,0.099010,0.029703,0.891089,0.079208,0.029703
Alaska,27.0,31.0,18.0,24.0,0.407407,0.407407,0.185185,0.444444,0.222222,0.333333,0.741935,0.193548,0.064516,0.838710,0.129032,0.032258
Atlantic,58.0,41.0,37.0,7.0,0.344828,0.310345,0.344828,0.241379,0.310345,0.448276,0.804878,0.195122,NaN,0.829268,0.170732,NaN
Brothers,17.0,40.0,13.0,1.0,0.294118,0.294118,0.411765,0.470588,0.176471,0.352941,0.800000,0.100000,0.100000,0.825000,0.150000,0.025000
Cats,31.0,73.0,22.0,1.0,0.419355,0.387097,0.193548,0.225806,0.451613,0.322581,0.013699,0.958904,0.027397,0.013699,0.958904,0.027397
Colorado,166.0,29.0,123.0,22.0,0.289157,0.343373,0.367470,0.271084,0.289157,0.439759,0.896552,0.103448,NaN,0.862069,0.137931,NaN
Cup,32.0,40.0,22.0,25.0,0.343750,0.312500,0.343750,0.406250,0.343750,0.250000,0.950000,0.050000,NaN,1.000000,NaN,NaN
Dance,14.0,68.0,9.0,NaN,0.285714,0.500000,0.214286,0.285714,0.428571,0.285714,0.073529,0.867647,0.058824,0.073529,0.882353,0.044118
Don't,571.0,35.0,437.0,NaN,0.178634,0.579685,0.241681,0.143608,0.607706,0.248687,NaN,1.000000,NaN,0.028571,0.971429,NaN
Eve,26.0,22.0,17.0,16.0,0.307692,0.346154,0.346154,0.307692,0.384615,0.307692,1.000000,NaN,NaN,1.000000,NaN,NaN


In [305]:
subset["count_train"].sum()

84373.0

In [325]:
sorted_sub = subset.sort_values("count_train", ascending=False)
sorted_sub.head(11)

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
sleeping,124.0,6874.0,0.741935,0.472505,0.290323,0.411290,0.298387,0.274194,0.362903,0.362903,0.066628,0.861944,0.071429,0.062264,0.866744,0.070992
outdoors,49.0,6406.0,0.673469,0.804090,0.367347,0.285714,0.346939,0.367347,0.326531,0.306122,0.157509,0.054480,0.788011,0.125351,0.055105,0.819544
cat,228.0,2917.0,0.627193,0.234487,0.385965,0.298246,0.315789,0.372807,0.385965,0.241228,0.114501,0.827563,0.057936,0.108330,0.832705,0.058965
no,13172.0,2693.0,0.808913,0.000743,0.131263,0.755694,0.113043,0.081840,0.770574,0.147586,0.101745,0.828073,0.070182,0.093947,0.836242,0.069811
alone,518.0,2295.0,0.691120,0.499782,0.245174,0.534749,0.220077,0.222008,0.542471,0.235521,0.108497,0.825708,0.065795,0.107190,0.833115,0.059695
Nobody,549.0,2269.0,0.819672,NaN,0.151184,0.774135,0.074681,0.040073,0.907104,0.052823,0.002644,0.996915,0.000441,NaN,1.000000,NaN
couch,35.0,2216.0,0.714286,0.731498,0.342857,0.371429,0.285714,0.485714,0.257143,0.257143,0.097022,0.799639,0.103339,0.093412,0.801895,0.104693
asleep,69.0,2116.0,0.579710,0.439981,0.246377,0.521739,0.231884,0.304348,0.463768,0.231884,0.095463,0.820416,0.084121,0.089792,0.824197,0.086011
new,3727.0,1944.0,0.702442,0.038580,0.412664,0.279045,0.308291,0.438154,0.246579,0.315267,0.765432,0.175412,0.059156,0.805556,0.160494,0.033951
competition,264.0,1601.0,0.742424,0.800750,0.390152,0.299242,0.310606,0.405303,0.257576,0.337121,0.809494,0.081199,0.109307,0.830106,0.071205,0.098688


In [294]:
subset["entailment_train_pred"] = subset["entailment_train_pred"].fillna(0)
subset["contradiction_train_pred"] = subset["contradiction_train_pred"].fillna(0)
subset["neutral_train_pred"] = subset["neutral_train_pred"].fillna(0)
subset["entailment_train_data"] = subset["entailment_train_data"].fillna(0)
subset["contradiction_train_data"] = subset["contradiction_train_data"].fillna(0)
subset["neutral_train_data"] = subset["neutral_train_data"].fillna(0)
subset = subset.dropna()
subset

C:\Users\ANSHUL~1\AppData\Local\Temp/ipykernel_13248/2007151226.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["entailment_train_pred"] = subset["entailment_train_pred"].fillna(0)
C:\Users\ANSHUL~1\AppData\Local\Temp/ipykernel_13248/2007151226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["contradiction_train_pred"] = subset["contradiction_train_pred"].fillna(0)
C:\Users\ANSHUL~1\AppData\Local\Temp/ipykernel_13248/2007151226.py:3: SettingWithCopyWarning: 
A value is trying to be set

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
4th,18.0,101.0,13.0,4.0,0.388889,0.111111,0.500000,0.388889,0.111111,0.500000,0.871287,0.099010,0.029703,0.891089,0.079208,0.029703
Alaska,27.0,31.0,18.0,24.0,0.407407,0.407407,0.185185,0.444444,0.222222,0.333333,0.741935,0.193548,0.064516,0.838710,0.129032,0.032258
Atlantic,58.0,41.0,37.0,7.0,0.344828,0.310345,0.344828,0.241379,0.310345,0.448276,0.804878,0.195122,0.000000,0.829268,0.170732,0.000000
Brothers,17.0,40.0,13.0,1.0,0.294118,0.294118,0.411765,0.470588,0.176471,0.352941,0.800000,0.100000,0.100000,0.825000,0.150000,0.025000
Cats,31.0,73.0,22.0,1.0,0.419355,0.387097,0.193548,0.225806,0.451613,0.322581,0.013699,0.958904,0.027397,0.013699,0.958904,0.027397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wishes,78.0,40.0,47.0,4.0,0.371795,0.320513,0.307692,0.512821,0.230769,0.256410,0.750000,0.200000,0.050000,0.850000,0.100000,0.050000
wolves,15.0,23.0,10.0,9.0,0.400000,0.400000,0.200000,0.200000,0.600000,0.200000,0.086957,0.913043,0.000000,0.086957,0.913043,0.000000
wrote,628.0,52.0,424.0,17.0,0.350318,0.251592,0.398089,0.464968,0.195860,0.339172,0.769231,0.192308,0.038462,0.807692,0.153846,0.038462
year,3324.0,227.0,2365.0,60.0,0.449759,0.266245,0.283995,0.466005,0.219615,0.314380,0.757709,0.193833,0.048458,0.810573,0.167401,0.022026


In [318]:
subset["word_counts_correct"] = subset["word_counts_correct"]/subset["count_val"]

C:\Users\ANSHUL~1\AppData\Local\Temp/ipykernel_13248/2202277948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["word_counts_correct"] = subset["word_counts_correct"]/subset["count_val"]


In [319]:
subset["word_counts_correct_train"] = subset["word_counts_correct_train"]/subset["count_train"]
subset.head(10)

C:\Users\ANSHUL~1\AppData\Local\Temp/ipykernel_13248/775974719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["word_counts_correct_train"] = subset["word_counts_correct_train"]/subset["count_train"]


,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
4th,18.0,101.0,0.722222,0.039604,0.388889,0.111111,0.500000,0.388889,0.111111,0.500000,0.871287,0.099010,0.029703,0.891089,0.079208,0.029703
Alaska,27.0,31.0,0.666667,0.774194,0.407407,0.407407,0.185185,0.444444,0.222222,0.333333,0.741935,0.193548,0.064516,0.838710,0.129032,0.032258
Atlantic,58.0,41.0,0.637931,0.170732,0.344828,0.310345,0.344828,0.241379,0.310345,0.448276,0.804878,0.195122,NaN,0.829268,0.170732,NaN
Brothers,17.0,40.0,0.764706,0.025000,0.294118,0.294118,0.411765,0.470588,0.176471,0.352941,0.800000,0.100000,0.100000,0.825000,0.150000,0.025000
Cats,31.0,73.0,0.709677,0.013699,0.419355,0.387097,0.193548,0.225806,0.451613,0.322581,0.013699,0.958904,0.027397,0.013699,0.958904,0.027397
Colorado,166.0,29.0,0.740964,0.758621,0.289157,0.343373,0.367470,0.271084,0.289157,0.439759,0.896552,0.103448,NaN,0.862069,0.137931,NaN
Cup,32.0,40.0,0.687500,0.625000,0.343750,0.312500,0.343750,0.406250,0.343750,0.250000,0.950000,0.050000,NaN,1.000000,NaN,NaN
Dance,14.0,68.0,0.642857,NaN,0.285714,0.500000,0.214286,0.285714,0.428571,0.285714,0.073529,0.867647,0.058824,0.073529,0.882353,0.044118
Don't,571.0,35.0,0.765324,NaN,0.178634,0.579685,0.241681,0.143608,0.607706,0.248687,NaN,1.000000,NaN,0.028571,0.971429,NaN
Eve,26.0,22.0,0.653846,0.727273,0.307692,0.346154,0.346154,0.307692,0.384615,0.307692,1.000000,NaN,NaN,1.000000,NaN,NaN


In [297]:
subset["word_counts_correct"].mean()

0.7007169532825909

In [298]:
subset["word_counts_correct_train"].mean()

0.36446365461104546

In [299]:
subset[subset["count_val"] > min_support]["word_counts_correct"].mean()

0.7006932810300871

In [300]:
subset[subset["count_val"] > min_support]

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred
Alaska,27.0,31.0,0.666667,0.774194,0.407407,0.407407,0.185185,0.444444,0.222222,0.333333,0.741935,0.193548,0.064516,0.838710,0.129032,0.032258
Atlantic,58.0,41.0,0.637931,0.170732,0.344828,0.310345,0.344828,0.241379,0.310345,0.448276,0.804878,0.195122,0.000000,0.829268,0.170732,0.000000
Cats,31.0,73.0,0.709677,0.013699,0.419355,0.387097,0.193548,0.225806,0.451613,0.322581,0.013699,0.958904,0.027397,0.013699,0.958904,0.027397
Colorado,166.0,29.0,0.740964,0.758621,0.289157,0.343373,0.367470,0.271084,0.289157,0.439759,0.896552,0.103448,0.000000,0.862069,0.137931,0.000000
Cup,32.0,40.0,0.687500,0.625000,0.343750,0.312500,0.343750,0.406250,0.343750,0.250000,0.950000,0.050000,0.000000,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
winning,153.0,684.0,0.640523,0.153509,0.477124,0.274510,0.248366,0.516340,0.209150,0.274510,0.899123,0.074561,0.026316,0.919591,0.067251,0.013158
wish,578.0,27.0,0.610727,0.259259,0.505190,0.271626,0.223183,0.474048,0.212803,0.313149,0.777778,0.222222,0.000000,0.814815,0.148148,0.037037
wishes,78.0,40.0,0.602564,0.100000,0.371795,0.320513,0.307692,0.512821,0.230769,0.256410,0.750000,0.200000,0.050000,0.850000,0.100000,0.050000
wrote,628.0,52.0,0.675159,0.326923,0.350318,0.251592,0.398089,0.464968,0.195860,0.339172,0.769231,0.192308,0.038462,0.807692,0.153846,0.038462


In [301]:
0.5269052574638183

0.5269052574638183

In [302]:
subset.loc[subset["entailment_pred"] > 0.5, "label"] = "contradiction"
subset.loc[subset["entailment_pred"] > 0.5, "prob"] = subset["entailment_pred"]

subset.loc[subset["contradiction_pred"] > 0.5, "label"] = "entailment"
subset.loc[subset["contradiction_pred"] > 0.5, "prob"] = subset["contradiction_pred"]

subset.loc[subset["neutral_pred"] > 0.5, "label"] = "neutral"
subset.loc[subset["neutral_pred"] > 0.5, "prob"] = subset["neutral_pred"]

# words = words.drop(["neutral_old", "contradiction_old", "entailment_old", "neutral_new", "contradiction_new", "entailment_new", "neutral_old_actual", "contradiction_old_actual", "entailment_old_actual", "neutral_new_actual", "contradiction_new_actual", "entailment_new_actual"], axis=1)
subset

C:\Users\Anshul Modh\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Anshul Modh\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\Anshul Modh\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1681: SettingWithCopyWarning: 
A value is trying to

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred,label,prob
4th,18.0,101.0,0.722222,0.039604,0.388889,0.111111,0.500000,0.388889,0.111111,0.500000,0.871287,0.099010,0.029703,0.891089,0.079208,0.029703,NaN,NaN
Alaska,27.0,31.0,0.666667,0.774194,0.407407,0.407407,0.185185,0.444444,0.222222,0.333333,0.741935,0.193548,0.064516,0.838710,0.129032,0.032258,NaN,NaN
Atlantic,58.0,41.0,0.637931,0.170732,0.344828,0.310345,0.344828,0.241379,0.310345,0.448276,0.804878,0.195122,0.000000,0.829268,0.170732,0.000000,NaN,NaN
Brothers,17.0,40.0,0.764706,0.025000,0.294118,0.294118,0.411765,0.470588,0.176471,0.352941,0.800000,0.100000,0.100000,0.825000,0.150000,0.025000,NaN,NaN
Cats,31.0,73.0,0.709677,0.013699,0.419355,0.387097,0.193548,0.225806,0.451613,0.322581,0.013699,0.958904,0.027397,0.013699,0.958904,0.027397,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wishes,78.0,40.0,0.602564,0.100000,0.371795,0.320513,0.307692,0.512821,0.230769,0.256410,0.750000,0.200000,0.050000,0.850000,0.100000,0.050000,neutral,0.512821
wolves,15.0,23.0,0.666667,0.391304,0.400000,0.400000,0.200000,0.200000,0.600000,0.200000,0.086957,0.913043,0.000000,0.086957,0.913043,0.000000,entailment,0.600000
wrote,628.0,52.0,0.675159,0.326923,0.350318,0.251592,0.398089,0.464968,0.195860,0.339172,0.769231,0.192308,0.038462,0.807692,0.153846,0.038462,NaN,NaN
year,3324.0,227.0,0.711492,0.264317,0.449759,0.266245,0.283995,0.466005,0.219615,0.314380,0.757709,0.193833,0.048458,0.810573,0.167401,0.022026,NaN,NaN


In [303]:
subset[subset["prob"] > 0.8]

,count_val,count_train,word_counts_correct,word_counts_correct_train,neutral_data,contradiction_data,entailment_data,neutral_pred,contradiction_pred,entailment_pred,neutral_train_data,contradiction_train_data,entailment_train_data,neutral_train_pred,contradiction_train_pred,entailment_train_pred,label,prob
frowning,13.0,260.0,0.923077,0.569231,0.153846,0.769231,0.076923,0.076923,0.846154,0.076923,0.023077,0.946154,0.030769,0.023077,0.942308,0.034615,entailment,0.846154
nowhere,176.0,49.0,0.835227,0.204082,0.130682,0.767045,0.102273,0.125000,0.806818,0.068182,0.163265,0.816327,0.020408,0.142857,0.857143,0.000000,entailment,0.806818
silent,259.0,135.0,0.861004,0.614815,0.100386,0.779923,0.119691,0.084942,0.822394,0.092664,0.051852,0.933333,0.014815,0.022222,0.977778,0.000000,entailment,0.822394
